# Mini-Challenge 1 : Train a Neural Network 

### Goals

Apply plain vanilla gradient descent to a relatively simple neural network. 
The network can be used to train a binary classification task such as distinguishing whether a cat is seen on a given image or not. 

For the implementation, a cost function is defined (on the training dataset) and the gradient w.r.t. the parameters of the model needs to be calculated. 

The network will be a fully connected neural network (multi-layer perceptron, MLP) with a single hidden layer. The implementation should be capable of dealing with differently shaped input datasets (input size of flattened image $n_0$) and number of units in the hidden layer ($n_1$). This will allow to apply your implementation to two different datasets, MNIST and a dataset with/without cats. 

**Overview**

| Problem | Content | Points |
| :--- | :--- | :---: |
| Problem 1 | Derive Formulas | 8 |
| Problem 2 | Implement Model and Cost Function and its Gradients | 8 |
| Problem 3 | Implement Gradient Descent Training | 8 |
| Problem 4 | Implement Gradient Checking | 6 |
| Problem 5 | Apply GD to MNIST | 4 |
| Problem 6 | Apply GD to Cats/NoCats | 4 |
| Total | | 38 |


Please send your solution per mail to [Martin](mailto:martin.melchior@fhnw.ch) and [Stefan](mailto:stefan.hackstein@fhnw.ch).


## Introductionary Remarks: Imports, Plotting

Implement the model as described below with <code>Numpy</code>. 
For plotting use <code>Matplotlib</code>. 

Hence, you will only need the following imports:


In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

from types import FunctionType ## this is only used to indicate the expected type of function arguments

For tracking progress of the optimization, you will plot the cost as a function of the iteration (or "epoch"). For this purpose, you can use the method <code>learningcurve_plots</code>. 

In [2]:
def learningcurve_plots(cost_hist: np.array, learning_speed_hist: np.array, logy: bool = False):
    """
    cost_hist -- history of cost values, as numpy-array of shape (T,1)
    learning_speed_hist -- history of learning speed values, as numpy-array of shape (T,1)
    logy -- if set to True will plot the y axis at logarithmic scale
    """
    plt.figure(1)
    T = len(cost_hist)
    if logy:
        plt.semilogy(np.arange(T),cost_hist,'b-')
    else:
        plt.plot(np.arange(T),cost_hist,'b-')
    plt.title("Cost")
        
    plt.figure(2)
    T = len(learning_speed_hist)
    if logy:
        plt.semilogy(np.arange(T),learning_speed_hist,'g-')
    else:
        plt.plot(np.arange(T),learning_speed_hist,'g-')
    plt.title("Learning Speed")

## Problem 1 : Derive Formulas for Model and Cost Function and Gradient

### Problem 1a): Model Function and Gradient

Here, you should mathematically describe the model function and its gradient. 
Actually, I will give you the structure of the model, you will then write down the components of the gradients.

As described above, the model contains a single hidden layer with $n^{(h)}$ units and a single input is given by an array of length $n^{(in)}$.
To compute the output from the input $x^{(in)}$, the following formulas hold:

$a^{(h)} = \sigma\left(W^{(h)}\cdot x^{(in)} + b^{(h)}\right)$<br>
$a^{(out)} = \sigma\left(W^{(out)}\cdot a^{(h)} + b^{(out)}\right)$

where 
* $W^{(h)}$ is a $n^{(h)}\times n^{(in)}$-matrix, 
* $b^{(h)}$ is a $n^{(h)}\times 1$-matrix, 
* $W^{(out)}$ is a $1\times n^{(h)}$-matrix, 
* $b^{(out)}$ is a scalar 

and $\sigma(z)=\frac{1}{1+\exp(-z)}$ is the sigmoid function which is applied element-wise if the input is a vector or matrix. The output of the model $a^{(out)}$ is a scalar. Note that the model output is a function of the input $x$ and the model parameters summarized as $\theta$, i.e. 

$$a^{(out)}=y^{(pred)}(x^{(in)},\theta)$$

Derive the formulas for the partial derivatives with respect to the components of the parameters $\theta$, i.e. of the components of $W^{(h)}, b^{(h)}, W^{(out)}, b^{(out)}$.
Define a scheme that makes efficient use of intermediate results, i.e. make manifest the terms that can be re-used for computing the parameters of the output and the hidden layer as well as the terms that can be reused to compute the weights and the bias for the output or the hidden layer. 


$$
a^{(out)} = y^{(pred)}(x^{(in)}, \theta)
$$

$$a^{(out)}=y^{(pred)}(x^{(in)}, W^{(h)}, b^{(h)}, W^{(out)}, b^{(out)})$$

$$
a^{({out})}=y^{(pred)}(x^{(in)},\theta)=\sigma\left( W^{{(out)}} \cdot
\sigma\left(W^{(h)} x^{(\text{in})}+b^{{(h)}}\right) +b^{{(out)}}\right)
$$

$$
\frac{\partial y^{({pred})}}{\partial x^{({in})}}=
\sigma\left(\sigma (W^{(h)} x^{({in})} + b^{({h})}) \cdot 
W^{({out})} + b^{({out})}\right) \cdot
\left(1 - \sigma(\sigma (W^{(h)} x^{({in})} + b^{({h})}) \cdot 
W^{({out})} + b^{({out})})\right)
$$

$$
\sigma\left(W^{(h)} x^{({in})}+b^{(h)}\right) \cdot \left(1 - \sigma(W^{(h)} x^{({in})}+b^{(h)})\right) \cdot
W^{{(h)}} \cdot W^{{(out)}}
$$

$$
\frac{\partial y^{({pred})}}{\partial x^{({in})}}= y^{({pred})} \cdot (1 - y^{({pred})}) \cdot
a^{({h})} \cdot (1 - a^{({h})}) \cdot W^{({h})} \cdot W^{({out})}
$$

$$
\frac{\partial y^{({pred})}}{\partial W^{({h})}}=
\sigma^{\prime}\left(\sigma (W^{(h)} x^{({in})} + b^{({h})}) \cdot W^{({out})} + b^{({out})}\right) \cdot \sigma^{\prime}\left(W^{(h)} x^{({in})}+b^{(h)}\right) \cdot x^{{(in)}} \cdot W^{{(out)}}
$$

$$
\frac{\partial y^{({pred})}}{\partial W^{({h})}}= y^{({pred})} \cdot (1 - y^{({pred})}) \cdot W^{({out})} \cdot x^{({in})} \cdot a^{({h})} \cdot (1 - a^{({h})})
$$

$$
\frac{\partial y^{({pred})}}{\partial b^{({h})}}=
\sigma^{\prime}\left(\sigma (W^{(h)} x^{({in})} + b^{({h})}) \cdot W^{({out})} + b^{({out})}\right) \cdot \sigma^{\prime}\left(W^{(h)} x^{({in})}+b^{(h)}\right) \cdot W^{{(out)}}
$$

$$
\frac{\partial y^{({pred})}}{\partial W^{({out})}}=
\sigma^{\prime}\left(\sigma (W^{(h)} x^{({in})} + b^{({h})}) \cdot W^{({out})} + b^{({out})}\right) \cdot \sigma\left(W^{(h)} x^{({in})}+b^{(h)}\right)
$$

$$
\frac{\partial y^{({pred})}}{\partial b^{({out})}}=
\sigma^{\prime}\left(\sigma (W^{(h)} x^{({in})} + b^{({h})}) \cdot W^{({out})} + b^{({out})}\right)
$$

### Probem 1b): Cost Function and Derivatives

As cost function, we use the binary cross-entropy loss applied to the model prediction $y^{(pred)}=a^{(out)}$ defined to be

$$L(y^{(pred)}, y) = -y\log(y^{(pred)})-(1-y)\log(1-y^{(pred)})$$

Here, $y$ denotes the binary label associated with the input, i.e. $y\in\{0,1\}$ (e.g. '0' - image contains not cat | '1' - image contains cat). For a given input $x^{(in)}=x$ we use the model with parameters $\theta$ to predict the output $a^{(out)}=y^{(pred)}(x,\theta)$ and then compute the loss according to the formula above. 

The cost function is defined as the loss function averaged over all the dataset, i.e.

$$C(\theta) = \frac{1}{N}\sum_{i=1}^N\,L\left(y^{(pred)}(x^{(i)},\theta), y^{(i)}\right)$$

We now use gradient descent to determine the parameters at which the cost function becomes minimal.

When computing the gradient of this cost w.r.t. model parameters, you need to compute the derivative of the cross entropy loss function, $L$ w.r.t. $y^{(pred)}$  and apply the chain rule.

Derive the formula for the derivative of $L$ w.r.t. $y^{(pred)}$.

---

$$\frac{\partial L{}}{\partial y^{({pred})}} =  -y\cdot\frac{y^{\prime\space{({pred})}}}{y^{({pred})}} - {({1-y})} \cdot \frac{-y^{\prime\space{({pred})}}}{1-y^{({pred})}}
$$

$$\frac{\partial L{}}{\partial y^{({pred})}} =  -y\cdot\frac{y^{({pred})}\cdot(1-y^{({pred})})}{y^{({pred})}} - {({1-y})} \cdot \frac{-y^{({pred})}\cdot(1-y^{({pred})})}{1-y^{({pred})}}$$

$$\frac{\partial L{}}{\partial y^{({pred})}} =  -y\cdot(1-y^{({pred})}) - {({1-y})} \cdot (-y^{({pred})})$$

Sofern $\log$ = $\ln$

---

## Problem 2 : Implement Model and Cost Function and its Gradient

Here, you translate the formulas of problem 1 into code. 

### Problem 2a): Model

For convenience, we define the model as a python class which allows to keep the parameters and intermediate results (such as $a^{(h)}$) as member variables.

The class should have the following instance methods:
* $forwardprop(x)$ : Compute the model output when passing an input $x$ through the model. Implement it such that you can handle multiple inputs at the same time, i.e. assume $x$ is a numpy array of shape $(n^{(in)},N)$ where $N$ is the number of inputs.
* $backwardprop(\dots)$ : Compute the gradient with respect to the model parameters. 
* $initialize(\dots)$ : Set initial values for the model parameters.
* $update\_params(\dots)$: Update the model parameters in accordance with the gradient descent update rule.

Make efficient use of the scheme derived in problem 1 by keeping intermediate results as member variables. Assume that $backwardprop$ is called after $forwardprop$ has been invoked. Introduce suitable member variables for the intermediate variables needed for the computation of the gradients. Furthermore, introduce member variables for the parameters (weights, bias) and their gradients.  

Assume the following shapes for the quantities involved ($n^{(in)}=n0, n^{(h)}=n1$):
* $W^{(h)}$ : 2d numpy array of shape (n1,n0)
* $b^{(h)}$ : 1d numpy array of shape (n1,1)
* $W^{(out)}$ : 1d numpy array of shape (1,n1)
* $b^{(out)}$ : scalar quantity 
* $x^{(in)}$ : 2d numpy array of shape (n0,N)
* $a^{(out)}$ : 2d numpy array of shape (1,N)

For the gradients, there will be a contribution for each input which needs to be properly averaged. The resulting gradient should have the same dimension as the parameter arrays. The input to the $backwardprop$ function is fed by the derivative of the loss function (evaluated element-wise per sample). This will allow to compute gradients with respect to the parameters properly averaged over all the samples. 


In [3]:
class Model:

    def __init__(self, ninput: int, nhidden: int):
        """
        Constructor of the model designed for given input dimension and number of units in hidden layer.
        
        Arguments:
        ninput - size of input
        nhidden - size of hidden layer
        """
        self.ninput = ninput
        self.nhidden = nhidden
        
    def _sigmoid(self, z: np.array) -> np.array2string:
        return 1.0 / (1.0 + np.exp(-z))
    
    def initialize(self, initial_weights_hidden: np.array, initial_weights_output: np.array):
        """
        Initialize the weights of the model with given input arguments and reset the biases to zero values. 
        
        Arguments:
        initial_weights_hidden - initial weights of the hidden layer (must be of shape (nhidden, ninput))
        initial_weights_output - initial weights of output layer (must be of shape (1, nhidden))
        """
        assert (self.nhidden,self.ninput)==initial_weights_hidden.shape
        assert (1,self.nhidden)==initial_weights_output.shape
        self.weights_hidden = initial_weights_hidden
        self.weights_output = initial_weights_output
        self.bias_hidden = np.zeros((self.nhidden,1), dtype=float)
        self.bias_output = 0.0
        
    def forwardprop(self, x: np.array) -> np.array:
        """
        Computes the forward path through the model. Keeps as state variables the inputs and the activations of the hidden layer and the output layer 
        (as numpy arrays of shape (n0,N), (n1,N), (1,N)). The activations of the layer are finally returned from the method call.
        
        Arguments:
        x - shape (n0, N)
        
        Returns:
        output of the model, numpy array of shape (1,N)
        """
        self.x = x
        self.a_h = self._sigmoid(self.weights_hidden @ x + self.bias_hidden)
        self.a_out = self._sigmoid(self.weights_output @ self.a_h + self.bias_output)
        return self.a_out

    def backwardprop(self, deriv_loss: np.array):
        """
        The computation is done after running the forwardprop. It keeps the gradient components with respect to the different 
        parameters in suitable arrays: 
        - grad_weights_hidden
        - grad_bias_hidden
        - grad_weights_output
        - grad_bias_output
        Note by passing in the derivate with respect to the loss it allows to compute the components of the gradient of the loss.
        
        Arguments:
        deriv_loss - the gradient of the loss w.r.t. output of the model, a numpy array of shape (1,N)
        
        Returns:
        None
        
        """
    def backwardprop(self, deriv_loss: np.array):
        self.grad_bias_output    = np.mean(deriv_loss * (self.a_out * (1-self.a_out)))
        self.grad_weights_output = deriv_loss @ self.a_out.T * ((1-self.a_out) @ self.a_h.T)
        self.grad_bias_hidden    = np.mean(deriv_loss * (self.a_out * (1-self.a_out))) * (self.grad_weights_output @ (self.a_h @ (1-self.a_h).T)).T 
        self.grad_weights_hidden = deriv_loss * self.a_out * (1-self.a_out) * (self.weights_output * (self.a_h * (1-self.a_h)).T).T @ self.x.T



    def update_params(self, lr: float):
        """
        Updates the parameters of the model according to gradient descent. It is assumed that the weights are properly initialized and the gradients
        previously computed.
        
        Arguments:
        lr - learning rate
        
        Returns:
        None
        
        """
        # START YOUR CODE
        self.weights_hidden -= lr * self.grad_weights_hidden
        self.weights_output -= lr * self.grad_weights_output
        self.bias_hidden -= lr * self.grad_bias_hidden
        self.bias_output -= lr * self.grad_bias_output
        # END YOUR CODE
            

### Test

In [4]:
# Unit test - following the convention that
# shape of gradients w.r.t. weights/biases = shape of weights/biases
 
# Test with a tiny model and one sample
mtest = Model(2,3)
whidden = np.array([[0.1,0.9],[-0.2,0.7],[1.1,-1]]).reshape(3,2)
woutput = np.array([[0.5,-0.3,.1],]).reshape(1,3)
 
# sample
x = np.array([1,1]).reshape(2,1)
 
# initialize
mtest.initialize(whidden, woutput)
np.testing.assert_array_almost_equal_nulp(whidden, mtest.weights_hidden)
np.testing.assert_array_almost_equal_nulp(woutput, mtest.weights_output)
 
# forward
np.testing.assert_almost_equal(0.5575659, mtest.forwardprop(x), decimal=7)
 
# backward  ## use convention: gradient.shape == parameter.shape
mtest.backwardprop(np.array([1,]).reshape(1,1))
 
# shapes
np.testing.assert_allclose((1,3), mtest.grad_weights_output.shape) # woutput.shape=(1,3)
#mtest.grad_bias_output -> scalar value
np.testing.assert_allclose((3,2), mtest.grad_weights_hidden.shape) # whidden.shape=(3,2)
np.testing.assert_allclose((3,1), mtest.grad_bias_hidden.shape) # bias_hidden = (3,1)
 
# regression results for the gradients
np.testing.assert_allclose(0.24668616, mtest.grad_bias_output)
np.testing.assert_array_almost_equal(np.array([0.18034203,0.1535521,0.1295051]).reshape(1,3), mtest.grad_weights_output, decimal=7)
np.testing.assert_array_almost_equal(np.array([0.02425072,-0.01739165,0.00615176]).reshape(3,1), mtest.grad_bias_hidden, decimal=7)
np.testing.assert_array_almost_equal(np.array([[0.02425072,-0.01739165,0.00615176],[0.02425072,-0.01739165,0.00615176]]).T.reshape(3,2), mtest.grad_weights_hidden, decimal=7)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 2 is different from 1)

### Problem 2b): Cost, Loss 

Similarly, we define a class for the Cross-Entropy loss function. While the loss has been defined in 1b), the **cost** just takes the average over all data points in the dataset used for training.

Implement for the cross-entropy class the instance methods 
* $cost(...)$
* $derivative\_loss(...)$

as described below. Note that these are evaluated for given numpy arrays $ypred$ and $ytrue$ with the model prediction $a^{(out)}$ and the true label $y$, each of shape $(1,N)$. 

In [ ]:
class CrossEntropy:

    def cost(self, ypred: np.array, ytrue: np.array) -> float:
        """
        Computes the average binary cross entropy cost for given arrays ytrue and ypred, with the ground truth label values (in {0,1}) 
        and the predicted proabability to observe label 1, respectively.  

        Arguments:
        ypred - numpy array of shape (1,N) with the predicted values for N data points
        ytrue - numpy array of shape (1,N) with the ground truth values for N data points

        Returns:
        Cost averaged over the samples, i.e. a scalar value.
        """
        # START YOUR CODE
        return -1/ytrue.shape[1] * np.sum(ytrue * np.log(ypred) + (1-ytrue) * np.log(1-ypred))
        # END YOUR CODE


    def derivative_loss(self, ypred: np.array, ytrue: np.array) -> np.array:
        """
        Computes the derivative of the cross entropy loss function for given true label y and predicted ypred. 
        We expect numpy arrays to be passed as input arguments, so that the derivative should be computed element-wise. 

        Arguments:
        ypred - numpy array of shape (1,N) with the predicted values for N data points
        ytrue - numpy array of shape (1,N) with the ground truth values for N data points

        Returns:
        Element-wise derivative of the loss function (numpy array of shape (1,N))
        """    
        # START YOUR CODE
        return -ytrue * (1-ypred) + (1-ytrue) * ypred
        # END YOUR CODE



In [ ]:
# Test
print(CrossEntropy().cost(np.array([0.1,0.9]).reshape(1,2), np.array([0,1]).reshape(1,2)))

derivative_loss = CrossEntropy().derivative_loss(np.array([0.1,0.9]).reshape(1,2), np.array([0,1]).reshape(1,2))
print(derivative_loss)
np.testing.assert_allclose((1,2), derivative_loss.shape)

## Problem 3: Implement the Gradient Descent Training (GD)

Implement the gradient descent update loop for a given instance of the model and cost.

In order to monitor the update of the training, keep book at each iteration about 
* value of the cost (at the given parameters)
* learning speed (~length of the gradient) 

In [ ]:
def vanilla_gradient_descent(model: Model,
                             cost: FunctionType, 
                             weights_hidden: np.array,
                             weights_output: np.array, 
                             data: dict, 
                             lr: float, 
                             epochs: int
                             ) -> tuple:
    """
    Performs plain vanilla gradient descent for the cost function. The variables x of the function are provided as np-arrays.
    
    Arguments:
    model -- model
    cost -- cost function object
    weights_hidden -- (initial) weights in the hidden layer
    weights_output -- (initial) weights in the output layer
    lr -- learning rate
    epochs -- maximum number of iterations
    data -- dataset as dictionary with the input data (key: "input" | value: numpy array of shape (n0,N)) and label data (key: "label" | numpy array of shape (1,N)) 
    
    data = {
        "label": shape ( 1,N),
        "input": shape (n0,N)
    }

    Returns:
    cost_hist -- history of the values of the cost function seen during the iteration loop: np-array of shape (T,1) where T is the number of iteration needed.
    learning_speed_hist -- history of the learning speed where the learning speed given by the norm of the difference between subsequent x-values: np-array of shape (T,1) where T is the number of iteration needed.
    """
    # START YOUR CODE
    data = np.load("./mnist.npz")
    X = data["X"].T[1:,:]
    y = data["y"].T.reshape(1, X.shape[1])
    initial_weights = utils.initial_weights(X.shape[0], 50)
    weights_hidden = initial_weights[0]
    weights_output = initial_weights[1]
    data_dict = {"label": "", "input": ""}
    data_dict["label"], data_dict["input"] = y, X
    def vanilla_gradient_descent(model, cost_obj, weights_hidden: np.array, weights_output: np.array, data: dict, lr: float, epochs: int) -> tuple:
        model.initialize(weights_hidden, weights_output)
        N = len(data["input"])
        for i in range(epochs):
            y_pred = model.forwardprop(data["input"])
            cost = cost_obj.cost(y_pred, data["label"])
            derive_loss = cost_obj.derivative_loss(y_pred, data["label"])
            model.backwardprop(derive_loss)
            model.update_params(lr)
            print("iteration {}, cost {}".format(i, cost))
    print(vanilla_gradient_descent(Model(64, 50), CrossEntropy(), weights_hidden, weights_output, data_dict, 0.001, 1000))

            

    # END YOUR CODE
    


The following helper function will be useful to initialize the weights arrays - that will be passed to the $initialize$-method of the <code>Model</code> class.  

In [ ]:
def initial_weights(n0: int, n1: int):
    """
    Prepares a random 2d numpy array of shape (n0,n1) suited as initial weights for the training of the neural network model.
    """
    
    weights_hidden = np.random.normal(size=(n1,n0), loc=0.0, scale=1.0/np.sqrt(n1*n0))
    weights_output = np.random.normal(size=(1,n1), loc=0.0, scale=1.0/np.sqrt(n1))
    return weights_hidden, weights_output

## Problem 4: Gradient Checking

Implement the function `numerical_gradient` which computes the numerical gradient based on a small step size $\varepsilon$. Use the formula 

$$
\frac{\partial f(x_1,\dots,x_n)}{\partial x_k} \approx \frac{f(x_1,\dots,x_k+\varepsilon,\dots,x_n) - f(x_1,\dots,x_k-\varepsilon,\dots,x_n)}{2\varepsilon}.
$$

Using the class implemented above, you have access to 
* your implementation of the gradient $\frac{\partial y^{\rm (pred)}(x,\Theta)}{\partial \Theta_k}$ 
* the model function $y^{\rm (pred)}(x,\Theta)$ that you can evaluate at different parameter values. Individually vary all single components by the parameter `eps`.

You can set the parameter values by using the initialize method - by restricting to the _weights_ parameters of the model. This together with the `forwardprop` allows you to compute the model output $a^{\rm (out)}$ for given parameters and $x$. Thereafter, you can compute  the derivatives with respect to the weights by using `backwardprop`. Note that, as $x$-values, you can use some arbitrary samples (random generated or from MNIST - see below). As derivatives w.r.t. to cost you can pass in just 1's.

For which values of `eps` do we obtain a resonable numerical approxmiation of the gradient? Test different (not too large values) and use `matplotlib` for determining a resonable `eps`.  Compute the norm of the the difference vector between the numerical approximation and the analytically computed gradient vector (by using the rules of calculus). 

In [ ]:
def numerical_gradient(model: Model,
                       x: np.array,
                       weights_hidden: np.array,
                       weights_output: np.array,
                       eps: float = 1e-5 
                       ) -> tuple:
    """
    Compute for the given model, data point (x,y) and weights parameters (biases equal to zero) a numerical gradient approximation of the 
    gradient. 
    
    Arguments:
    model -- instance of the model class defined above
    x -- data point (np.array with shape (ninput,))
    weights_hidden -- weights matrix of shape (nhidden, ninput)
    weights_output -- weights matrix of shape (noutput, nhidden)
    eps -- value of epsilon (float)
    
    Returns:
    gradient -- numerical gradient at the given weights parameters (for the hidden and the output layer)
    """
    # START YOUR CODE



    # END YOUR CODE

In [ ]:
# Comparison with the analytically computed gradient vector

# START YOUR CODE



# END YOUR CODE

## MNIST Dataset

We here want to train a classifier for the MNIST dataset that is often used in machine learning. The images are digitized, hand-written figures. Actually, we use a light-weight version which consists of only 1797 images with 8x8 pixels. The vectors contained in $X$ are 64-dim vectors (8x8=64) - its elements correspond to the gray-scale values of the pixels (normalized to be included in [0,1]). 

The label values are set to '1' (`true`) if the hand-written figure is a '1' and '0' (`false`) in all the other cases. 

In [ ]:
# Please make sure that the data file is included in the same folder as the given jupyter notebook. 
data = np.load( "./mnist.npz" )
X = data['X'].T[1:,:]
y = data['y'].T.reshape(1,X.shape[1])
print(X.shape,y.shape)

Now, you can access the samples of the dataset by accessing the columns of $X$. In numpy: `x_i = X[:,i]`. 
If you reshape it to 8x8 you can eventually recognize the digit depicted on the image (a '0' in the example below):

In [ ]:
print(X[:,0])
print(y)

The 2d arrays can be plotted as image by applying the following code snippets: 

In [ ]:
item = 0
plt.imshow( X[:,item].reshape(8,8), cmap="binary" )
plt.title( "Label: " + str( y[0,item] )) 

The following should rather be a '1' (since the label is '1'):

In [ ]:
item = 1
plt.imshow( X[:,item].reshape(8,8), cmap="binary" )
plt.title( "Label: " + str( y[0,item] )) 

## Problem 5: Apply GD to MNIST

Apply the gradient descent to the model defined above. We here ignore ML specific aspects such as using a part of the data 
for validation and test or dealing with class-imbalance. We rather just want to see whether the optimization works.  

(1) Apply the functionality implemented above to train MNIST. Use the <code>learningcurve_plots</code> function to visualize whether gradient descent was performing well over the iterations. Also compute the _final error rate_, i.e. the percentage of wrong predictions. The predicted label can be obtained from $y^{(pred)}(\theta,x)$ by just rounding it to 0 or 1.

(2) Compare and characterize the shape of the learning for different learning rates. What's the "best" learning rate (<code>lr</code>). How many iterations (<code>epochs</code>) should be set?

In [ ]:
n0 = X.shape[0]
n1 = 50

lr = 6
epochs = 2000
print("Learning Rate: %6.5f"%lr)
print("Epochs: %6.5f"%epochs)

# START YOUR CODE


# END YOUR CODE


# Cat/NoCat Images

You now can apply the approach(es) implemented above to the cat/nocat use case - alternatively, you can pick another suitable use case. 

For loading the cat/nocat dataset run the following lines of code. 

In [ ]:
cat_nocat_data = np.load("./cat_nocat.npz") # point to the suitable folder relative to the folder of the notebook
X_train = cat_nocat_data['X_train'].T
X_test = cat_nocat_data['X_test'].T
y_train = cat_nocat_data['y_train'].T.reshape(1,X_train.shape[1])
y_test = cat_nocat_data['y_test'].T.reshape(1,X_test.shape[1])
[X_train.shape, y_train.shape, X_test.shape, y_test.shape]

With the following function you can plot a few sample images:

In [ ]:
def plot_image( X, y ):
    plt.imshow( X.reshape(64,64,3) )
    title = 'cat' if y==1 else 'non-cat'
    plt.title( title )

plt.figure(figsize=(20,20))
for i in range(10):
    for j in range(10):
        plt.subplot2grid((10,10),(i,j))
        plot_image( X_train[:,10*i+j], y_train[0,10*i+j] )
plt.tight_layout()

Actually, we have loaded 
* a training set (with 209 samples) that can be used to train the model, i.e. optimize the model parameters
* a test set (with 50 samples) that can be used to test the performance of the trained model.

The images are flattened arrays of size $64\times 64\times 3=12288$, i.e. quite large arrays.

## Problem 6 : Apply GD to Cat/NoCat Dataset

Apply Gradient Descent to train a classifier that can distinguish images with or without cats. Reuse the functionality implemented in Problem 5.

(1) Optimize the defined cost function with `X_train` und `y_train`. Again, use <code>learningcurve_plots</code> to visualize whether gradient descent was performing well over the iterations. The training will take more time than for MNIST. So, at beginning use at max 1000 Iterationen.

(2) Use the trained model to make predictions for the test data (not used for training). Determine the percentage of correct predictions for the train and test dataset (alternatively the error rate). As label predicted by the model you can use (for simplicity) $y^{(pred)}(\theta, x)$ round to 0 or 1. 

(3) What is a reasonable number of iterations (epochs) to be chosen?

(4) Explore the images for which the model gives a wrong prediction.

In [ ]:
n0 = X_train.shape[0]
n1 = 50

lr = 0.02
epochs = 1000
print("Learning Rate: %6.5f"%lr)
print("Epochs: %6.5f"%epochs)


# START YOUR CODE



# END YOUR CODE


### Inspect where Classifier is confused

i.e. where the trained classifier makes the wrong prediction.


In [ ]:
ypred = model.forwardprop(X_test)
indices = (np.round(ypred).astype(int) != y_test)[0,:]
confused = X_test[:,indices]
label = y_test[:,indices]

In [ ]:
plt.figure(figsize=(20,20))
for i in range(4):
    for j in range(4):
        plt.subplot2grid((10,10),(i,j))
        plot_image( confused[:,4*i+j], label[0,4*i+j] )
plt.tight_layout()

In [ ]:
ypred = model.forwardprop(X_test)
indices = (np.round(ypred).astype(int) == y_test)[0,:]
correct = X_test[:,indices]
label = y_test[:,indices]

In [ ]:
plt.figure(figsize=(20,20))
for i in range(4):
    for j in range(4):
        plt.subplot2grid((10,10),(i,j))
        plot_image( correct[:,4*i+j], label[0,4*i+j] )
plt.tight_layout()